In [1]:
import simpy
import pandas as pd
import numpy as np 
#from collections import namedtuple
#from recordtype import recordtype
import random 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from prettytable import PrettyTable
#get_ipython().magic('matplotlib inline')
#from keras.models import Sequential
#from keras.layers import Dense, Dropout
#from keras.optimizers import Adam 
from collections import deque 
from itertools import product
import math
from DQN import DQN
        
def episode(env,DQN,Tsc,Tfu,allowed_actions): #pacing rate가 각 flow에게 할당
    global action
    global state
    global request
    global flow_success
    fs=request['filesize']
    cnt=1
    c=0
    dqn_agent = DQN()
    while True: #Simularion time 동안 episode를 반복한다
        print ("-------------")
        print("Episode start",cnt)
        print ("")
        first_action=1 #new action의 경우 학습데이터에 추가하지 않음
        
        #1개의 에피소드는 모든 filesize가 0이 될때까지 실행
        while ((request['filesize'][0]!=0)or(request['filesize'][1]!=0)or(request['filesize'][2]!=0)):#모두 전송이 완료될 때 까지
            c+=1
            try: #state = filesize/deadline
                state=[math.ceil(request['filesize'][s]/request['deadline'][s]) for s in range(len(request['sources']))]
            except:#deadline이 0인 source가 존재하는 경우 (0 division 에러)
                for s in range(len(sources)):#deadline이 0이 아닌 source는 그대로
                    if (request['deadline'][s] != 0):
                        state[s]= math.ceil(request['filesize'][s]/request['deadline'][s])
                    else:#deadline이 0인 source의 state는 0
                        state[s]=0

            if (first_action==1): #New state-action policy를 EDF방식으로 (가장 deadline이 시급한)     
                action=[0,0,0]
                index=np.argmin(request['deadline']) #deadline 최소인 source의 index
                action[index]=10 #bottleneck capacity
                
            else: 
                action,p=dqn_agent.choose_action(state,allowed_actions) #DQN에 의한 액션선택
                pexp.append(p)
                

            for i in range(Tsc):#Scheduling interval 시작
                for s in range(len(sources)): #각 source에 대해 반복
                    request['filesize'][s]=max([request['filesize'][s]-int(action[s]*1024/10),0]) #음수 X
                    request['deadline'][s]=max(request['deadline'][s]-1,0) #음수 X, deadline은 Tsc만큼 감소
                    # deadline을 양수로 해야하는 이유 > 음수면  value가 1로 할당됨
                    if ((request['filesize'][s]==0) and (request['value'][s]==None)): #아직 완료되지 않았던 flow가 전송이 완료되면?
                        #print ("s{}의 전송이 완료됨".format(s))
                        if (request['deadline'][s]!=0): #기간 안에 전송되면?
                            request['value'][s]=1 #value를 1로 변경
                            flow_success.append(1)
                            print ("s{}의 전송이 deadline 안에 완료됨".format(s))
                        else: #기간안에 전송된게 아니라면(value초기값은 None)
                            request['value'][s]=0
                            flow_success.append(0)
                            print ("s{}의 전송이 deadline을 지나 완료됨".format(s))
                yield env.timeout(Tfu)# Tfu(1초)마다 위 과정 실행
            
            
            print ("action", action)
#             print ("")
#             print ("source 0 전송할 file:",request['filesize'][0])
#             print ("")
#             print ("source 1 전송할 file:",request['filesize'][1])
#             print ("")
#             print ("source 2 전송할 file:",request['filesize'][2])
        
            #모든 전송이 완료된 후 next_state는 고려할 필요 없음: terminal=True로 하여 target에 reward를 할당
            if ((request['filesize'][0]==0)and(request['filesize'][1]==0)and(request['filesize'][2]==0)):
                terminal = True
            else : 
                terminal = False
            
            
            try: 
                next_state=[math.ceil(request['filesize'][s]/request['deadline'][s]) for s in range(len(request['sources']))]
            except:#deadline이 0인 경우 error
                for s in range(len(sources)):
                    if (request['deadline'][s] != 0):
                        next_state[s]= math.ceil(request['filesize'][s]/request['deadline'][s])
                    else:
                        next_state[s]=0
                        
            #print(next_state)
                        
            # build your reward function 리워드 방정식 입력
            reward = sum((np.max(request['deadline'])-request['deadline'])*action)
            cur_state = state 
            action = action
            new_state = next_state 
            #new_action = next_allowed_actions
            reward = reward
            terminal = terminal
            
            if (first_action==0): # 첫번쨰 선택 액션이 아닌경우에만 학습
                dqn_agent.remember(cur_state, action, reward, new_state,terminal) #새로운 state로 설정해주고 기존state저장
                mse_loss.append(dqn_agent.replay(allowed_actions))#학습, loss 저장
                dqn_agent.train_target()
            
            first_action=0 

        for i in range(len(sources)):
            if (request['value'][i]==1):
                print ("source {} 전송완료".format(i))
            else:
                print ("source {} deadine 충족하지 못함".format(i))
        
        print("모두 전송 완료")

        filesize=[random.randrange(10,50)*1024 for source in range(len(sources))] #단위 Mb
        deadline=[int(filesize[source]/307) for source in range(len(sources))] #sum_rmin이  9
        request = {
            'sources' : sources,
            'destinations' : destinations, 
            'filesize' : filesize, #단위는 Gbps
            'deadline' : deadline,
            'value' : [None for source in sources] #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
        }#다음 episode를 위해 request 생성

        state=[0,0,0]#초기 state
        action=[0,0,0]#초기 action ->고칠것
        print ("episode end")
        print("-----------------")
        cnt+=1
    
    dqn_agent.save_model("policy_network.h5")
    
    


#한 번의 에피소드 프로세스
#N개의 source에서 동시에 request를 발생시키면 시작됨
#끝나는 기점은 모든 flow가 전송이 완료되는 시점
#랜덤으로 filesize를 생성, deadline도 랜덤으로 생성
env = simpy.Environment()

mse_loss=[] #episode의 진행에 따른 mse_loss의 변화율 graph
flow_success=[] #episode의 진행에 따른 flow_success rate
pexp=[] #action choice 진행에 따른 pexp 변화

state=[0,0,0]#초기 state
action=[0,0,0]#초기 action ->고칠것
Tsc=5 #scheduling interval을 구성하는 flow update interval의 수
Tfu=0.1 #flow update interval의 시간단위(0.1초로 가정)
sources = ['s0', 's1', 's2']
destinations = ['d0', 'd1', 'd2']
filesize=[random.randrange(10,50)*1024 for source in range(len(sources))] #단위 Mb
deadline=[int(filesize[source]/307) for source in range(len(sources))] #sum_rmin이  9
request = {
    'sources' : sources,
    'destinations' : destinations, 
    'filesize' : filesize, #단위는 Mbps
    'deadline' : deadline,
    'value' : [None for source in sources] #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
}

A=list(range(11))
B=list(range(11))
C=list(range(11)) 
allowed_actions=[] #합이 10이 되는 0~10까지의 수 조합
for i in list(product(*(A,B,C))):
    if sum(list(i))==10:
        allowed_actions.append(i) #d는 66개의 조합
        
#print ("action", action)
#print ("")
# print ("source 0 전송할 file:",request['filesize'][0])
# print ("")
# print ("source 1 전송할 file:",request['filesize'][1])
# print ("")
# print ("source 2 전송할 file:",request['filesize'][2])

# print ("source 0의 min_rate:",math.ceil(request['filesize'][0]/request['deadline'][0]))
# print("")
# print ("source 1의 min_rate:",math.ceil(request['filesize'][1]/request['deadline'][1]))
# print("")
# print ("source 0의 min_rate:",math.ceil(request['filesize'][2]/request['deadline'][2]))


env.process(episode(env,DQN,Tsc,Tfu,allowed_actions))
env.run(until=10000)#10만 초 동안 가동
print("종료")

#print (mse_loss)
#print (flow_success)


Using TensorFlow backend.
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

-------------
Episode start 1

action [0, 0, 10]
0.999
1
random action
action (9, 0, 1)
update weights
0.999
2
random action
action (4, 4, 2)
update weights
0.999
3
random action
action (0, 0, 10)
update weights
0.999
4
random action
action (6, 3, 1)
update weights
0.999
5
random action
action (5, 3, 2)
update weights
0.999
6
random action
action (1, 6, 3)
update weights
0.999
7
random action
action (2, 6, 2)
update weights
0.999
8
random action
action (5, 0, 5)
update weights
0.999
9
random action
action (2, 0, 8)
update weights
0.999
10
random action
action (6, 1, 3)
update weights
0.999
11
random action
action (5, 0, 5)
update weights
0.999
12
random action
action (2, 5, 3)
update weights
0.999
13
random action
action (4, 5, 1)
update weights
0.999
14
random action
action (6, 1, 3)
update weights
0.999
15
random action
action (5, 5, 0)
update weights
0.999
16
random action
action (6, 3, 1)
update weights
0.999
17
random action
action (6, 1, 3)
update weights
0.999
18
random action
a

update weights
0.999
131
random action
action (1, 0, 9)
update weights
0.999
132
random action
action (8, 1, 1)
update weights
0.999
133
random action
action (3, 2, 5)
update weights
0.999
134
random action
action (4, 3, 3)
update weights
0.999
135
random action
action (0, 7, 3)
update weights
0.999
136
random action
action (0, 8, 2)
update weights
0.999
137
random action
action (8, 0, 2)
update weights
0.999
138
random action
action (2, 2, 6)
update weights
0.999
139
random action
s2의 전송이 deadline 안에 완료됨
action (2, 2, 6)
update weights
0.999
140
random action
action (8, 0, 2)
update weights
0.999
141
random action
action (6, 4, 0)
update weights
0.999
142
random action
s1의 전송이 deadline을 지나 완료됨
action (0, 8, 2)
update weights
source 0 전송완료
source 1 deadine 충족하지 못함
source 2 전송완료
모두 전송 완료
episode end
-----------------
-------------
Episode start 6

action [0, 0, 10]
0.999
143
random action
action (0, 4, 6)
update weights
0.999
144
random action
action (3, 2, 5)
update weights
0.999
145
r

update weights
0.999
263
random action
action (6, 0, 4)
update weights
0.999
264
random action
action (10, 0, 0)
update weights
0.999
265
random action
action (3, 3, 4)
update weights
0.999
266
random action
s1의 전송이 deadline 안에 완료됨
action (3, 4, 3)
update weights
0.999
267
random action
action (0, 7, 3)
update weights
0.999
268
random action
action (7, 0, 3)
update weights
0.999
269
random action
action (4, 1, 5)
update weights
0.999
270
random action
action (6, 1, 3)
update weights
0.999
271
random action
action (0, 6, 4)
update weights
0.999
272
random action
s0의 전송이 deadline 안에 완료됨
action (9, 1, 0)
update weights
0.999
273
random action
action (3, 0, 7)
update weights
0.999
274
random action
action (2, 5, 3)
update weights
0.999
275
random action
action (6, 0, 4)
update weights
0.999
276
random action
action (3, 3, 4)
update weights
0.999
277
random action
s2의 전송이 deadline을 지나 완료됨
action (5, 1, 4)
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode e

update weights
0.999
388
random action
action (6, 2, 2)
update weights
0.999
389
random action
action (1, 5, 4)
update weights
0.999
390
random action
s1의 전송이 deadline 안에 완료됨
action (4, 4, 2)
update weights
0.999
391
random action
action (3, 1, 6)
update weights
0.999
392
random action
action (8, 1, 1)
update weights
0.999
393
random action
action (3, 7, 0)
update weights
0.999
394
random action
action (4, 5, 1)
update weights
0.999
395
random action
action (3, 6, 1)
update weights
0.999
396
random action
action (2, 8, 0)
update weights
0.999
397
random action
action (0, 3, 7)
update weights
0.999
398
random action
s2의 전송이 deadline을 지나 완료됨
action (6, 2, 2)
update weights
0.999
399
random action
action (10, 0, 0)
update weights
0.999
400
random action
action (5, 2, 3)
update weights
0.999
401
random action
s0의 전송이 deadline 안에 완료됨
action (10, 0, 0)
update weights
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
episode end
-----------------
-------------
Episode start 17

ac

...

update weights
0.01
8647
action (1, 0, 9)
update weights
0.01
8648
action (1, 0, 9)
update weights
0.01
8649
action (1, 0, 9)
update weights
0.01
8650
action (1, 0, 9)
update weights
0.01
8651
action (1, 0, 9)
update weights
0.01
8652
action (1, 0, 9)
update weights
0.01
8653
action (1, 0, 9)
update weights
0.01
8654
action (1, 0, 9)
update weights
0.01
8655
action (1, 0, 9)
update weights
0.01
8656
action (1, 0, 9)
update weights
0.01
8657
action (1, 0, 9)
update weights
0.01
8658
action (1, 0, 9)
update weights
0.01
8659
action (1, 0, 9)
update weights
0.01
8660
action (1, 0, 9)
update weights
0.01
8661
action (1, 0, 9)
update weights
0.01
8662
action (1, 0, 9)
update weights
0.01
8663
action (1, 0, 9)
update weights
0.01
8664
action (1, 0, 9)
update weights
0.01
8665
action (1, 0, 9)
update weights
0.01
8666
action (1, 0, 9)
update weights
0.01
8667
action (1, 0, 9)
update weights
0.01
8668
action (1, 0, 9)
update weights
0.01
8669
action (1, 0, 9)
update weights
0.01
8670
action (1

update weights
0.01
8842
action (1, 0, 9)
update weights
0.01
8843
action (1, 0, 9)
update weights
0.01
8844
action (1, 0, 9)
update weights
0.01
8845
action (1, 0, 9)
update weights
0.01
8846
action (1, 0, 9)
update weights
0.01
8847
action (1, 0, 9)
update weights
0.01
8848
action (1, 0, 9)
update weights
0.01
8849
action (1, 0, 9)
update weights
0.01
8850
action (1, 0, 9)
update weights
0.01
8851
action (1, 0, 9)
update weights
0.01
8852
action (1, 0, 9)
update weights
0.01
8853
action (1, 0, 9)
update weights
0.01
8854
action (1, 0, 9)
update weights
0.01
8855
action (1, 0, 9)
update weights
0.01
8856
action (1, 0, 9)
update weights
0.01
8857
action (1, 0, 9)
update weights
0.01
8858
action (1, 0, 9)
update weights
0.01
8859
action (1, 0, 9)
update weights
0.01
8860
action (1, 0, 9)
update weights
0.01
8861
action (1, 0, 9)
update weights
0.01
8862
action (1, 0, 9)
update weights
0.01
8863
action (1, 0, 9)
update weights
0.01
8864
action (1, 0, 9)
update weights
0.01
8865
action (1

update weights
0.01
9037
action (1, 0, 9)
update weights
0.01
9038
action (1, 0, 9)
update weights
0.01
9039
action (1, 0, 9)
update weights
0.01
9040
action (1, 0, 9)
update weights
0.01
9041
action (1, 0, 9)
update weights
0.01
9042
action (1, 0, 9)
update weights
0.01
9043
action (1, 0, 9)
update weights
0.01
9044
action (1, 0, 9)
update weights
0.01
9045
action (1, 0, 9)
update weights
0.01
9046
action (1, 0, 9)
update weights
0.01
9047
action (1, 0, 9)
update weights
0.01
9048
action (1, 0, 9)
update weights
0.01
9049
action (1, 0, 9)
update weights
0.01
9050
action (1, 0, 9)
update weights
0.01
9051
action (1, 0, 9)
update weights
0.01
9052
action (1, 0, 9)
update weights
0.01
9053
action (1, 0, 9)
update weights
0.01
9054
action (1, 0, 9)
update weights
0.01
9055
action (1, 0, 9)
update weights
0.01
9056
action (1, 0, 9)
update weights
0.01
9057
action (1, 0, 9)
update weights
0.01
9058
action (1, 0, 9)
update weights
0.01
9059
action (1, 0, 9)
update weights
0.01
9060
action (1

update weights
0.01
9233
action (1, 0, 9)
update weights
0.01
9234
action (1, 0, 9)
update weights
0.01
9235
action (1, 0, 9)
update weights
0.01
9236
action (1, 0, 9)
update weights
0.01
9237
action (1, 0, 9)
update weights
0.01
9238
action (1, 0, 9)
update weights
0.01
9239
action (1, 0, 9)
update weights
0.01
9240
action (1, 0, 9)
update weights
0.01
9241
action (1, 0, 9)
update weights
0.01
9242
action (1, 0, 9)
update weights
0.01
9243
action (1, 0, 9)
update weights
0.01
9244
action (1, 0, 9)
update weights
0.01
9245
action (1, 0, 9)
update weights
0.01
9246
action (1, 0, 9)
update weights
0.01
9247
action (1, 0, 9)
update weights
0.01
9248
action (1, 0, 9)
update weights
0.01
9249
action (1, 0, 9)
update weights
0.01
9250
action (1, 0, 9)
update weights
0.01
9251
action (1, 0, 9)
update weights
0.01
9252
action (1, 0, 9)
update weights
0.01
9253
action (1, 0, 9)
update weights
0.01
9254
action (1, 0, 9)
update weights
0.01
9255
action (1, 0, 9)
update weights
0.01
9256
action (1

KeyboardInterrupt: 

이렇게 하나의 action에 수렴하는 이유가 혹시 target update 시 action_id에 할당하는 Q값때문이 아닌가..?

In [3]:
flow_success

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [22]:
rate=[]
flow_success=np.array(flow_success)
for i in range(int(len(flow_success)/3)):#200개당평균
    print(flow_success[3*i:3*(i+1)])
    rate.append(np.mean(flow_success[3*i:3*(i+1)]))
    print(np.mean(flow_success[3*i:3*(i+1)]))


[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[

0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 1]
1

1.0
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 1 1]
1.0
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 0 1]
0.6666666666666666
[1 1 1]
1.0
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 

0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 1 1]
1.0
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 

0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0

[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 1]
1.0
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 1]
1.0
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 1 1]
1.0
[1 1 1]
1.0
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 1]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 0 0]
0.3333333333333333
[1 0 0]
0.3333333333333333
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[1 1 0]
0.6666666666666666
[

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# x = mse_loss

# plt.plot(x)
# plt.show()

#flow_success_rate=
x2 = rate

plt.plot(x2)
plt.show()

KeyboardInterrupt: 

In [74]:
len(mse_loss)

1968

1.할 것 : flow update interval을 줄이기 (0.001초?) 어느정도가 적당할 지 교수님꼐여줘보기

2.reward의 문제점: drem(deadline까지 남아있는 시간)이 가장 짧은 source에 많은 pacing rate를 할당할 수록 높은 reward가 주어지는데, file size를 고려하지 않았다는 문제점이 있다. deadline이 아니라 min rate를 기준으로 해야할듯, 그리고 9:1과 같은 극단적비율로 학습됨 

3.state=0인 경우 filesize=0이란뜻이므로 action을 0으로 할당하는 코드를 짜야함. 문제는 두개의 filesize가 0인경우 어떻게 하냐는것...!!!!! -> state가 0인 index에 따라 action도 0인 allowed_action의 index를 찾고, 그 중 가장 큰 maxQ값을 갖는 action 선택 
bug: source2(3번째 소스)의 state가 0 이면 action을 3,3,4로 할당(원인은 모름)

4.10만초 동안 가동

5.deadline이 지나면 음수까지 내려가는 버전으로 돌려보기 > 이것은 안됨... value가 deadline이 0이면 0이되도록 설정했기때문에 deadline이 음수가 되면 value가 1이 됨



In [28]:
A=list(range(11))
B=list(range(11))
C=list(range(11)) #합이 10이 되는 0~10까지의 수 조합
allowed_actions=[]
for i in list(product(*(A,B,C))):
    if sum(list(i))==10:
        allowed_actions.append(i) #d는 66개의 조합
        
allowed_actions.index((0,10,0))
#print (allowed_actions)

10

In [12]:
state=np.array([3,3,3])
np.array(state).reshape(1,state.shape[0])

array([[3, 3, 3]])

In [82]:
#deeplearning없이 시뮬레이션
env = simpy.Environment()

request = {
    'sources' : sources,
    'destinations' : destinations, 
    'filesize' : [random.randrange(50,100) for source in sources], #단위는 Gbps
    'deadline' : [random.randrange(5,30) for source in sources], 
    'value' : [None for source in sources], #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
}

print ("source 0의 filesize:",request['filesize'][0])
print ("source 1의 filesize:",request['filesize'][1])
print ("source 2의 filesize:",request['filesize'][2])

action=[3,3,4]

Tfu=1

def flowUpdate_interval(env,action,Tfu): #Tsc에서의 action(pacint rate)가 n번의 flow update interval만큼 유지된다.
    #print("flow update interval :",env.now)
    global request
    fs=request['filesize']
    #src=request['sources']
    while True:
        print("flow update interval :",env.now)
        if ((fs[0]!=0)or(fs[1]!=0)or(fs[2]!=0)): #하나라도 0이상으로 남아있으면
            
            print("전송중!영차영차")
            
            for s in range(len(sources)):#0밑으로는 안떨어지게, 전역변수 request의 값을 변경
                request['filesize'][s]=max([request['filesize'][s]-action[s],0]) 
                request['deadline'][s]=max([request['deadline'][s]-1,0])
                if ((request['filesize'][s]==0) and (request['value'][s]==None)): #아직 완료되지 않았던 flow가 전송이 완료되면?
                    #print ("s{}의 전송이 완료됨".format(s))
                    if (request['deadline'][s]!=0): #기간 안에 전송되면?
                        request['value'][s]=1 #value를 1로 변경
                        print ("s{}의 전송이 deadline 안에 완료됨".format(s))
                    else: #기간안에 전송된게 아니라면(value초기값은 None)
                        request['value'][s]=0
                        print ("s{}의 전송이 deadline을 지나 완료됨".format(s))
            
            yield env.timeout(Tfu) 
        elif ((fs[0]==0)and(fs[1]==0)and(fs[2]==0)):
            print ("종료:",env.now)
            return #모든 flow의 전송이 완료되면 종료
        
        
proc=env.process(flowUpdate_interval(env,action,Tfu))
env.run(until=proc)

source 0의 filesize: 87
source 1의 filesize: 56
source 2의 filesize: 78
flow update interval : 0
전송중!영차영차
flow update interval : 1
전송중!영차영차
flow update interval : 2
전송중!영차영차
flow update interval : 3
전송중!영차영차
flow update interval : 4
전송중!영차영차
flow update interval : 5
전송중!영차영차
flow update interval : 6
전송중!영차영차
flow update interval : 7
전송중!영차영차
flow update interval : 8
전송중!영차영차
flow update interval : 9
전송중!영차영차
flow update interval : 10
전송중!영차영차
flow update interval : 11
전송중!영차영차
flow update interval : 12
전송중!영차영차
flow update interval : 13
전송중!영차영차
flow update interval : 14
전송중!영차영차
flow update interval : 15
전송중!영차영차
flow update interval : 16
전송중!영차영차
flow update interval : 17
전송중!영차영차
flow update interval : 18
전송중!영차영차
s1의 전송이 deadline 안에 완료됨
flow update interval : 19
전송중!영차영차
s2의 전송이 deadline 안에 완료됨
flow update interval : 20
전송중!영차영차
flow update interval : 21
전송중!영차영차
flow update interval : 22
전송중!영차영차
flow update interval : 23
전송중!영차영차
flow update interval : 24
전송중!영차영차
flow update interv